In [2]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import pickle

## Data

In [3]:
file = open('data_pickles/processed_features_plus', 'rb')
X_train, Y_train, X_test, Y_test, X_val, Y_val = pickle.load(file)
file.close()

In [4]:
reshape_form = X_train.shape[1]

X_train = X_train.reshape(len(X_train), reshape_form)
X_test = X_test.reshape(len(X_test), reshape_form)
X_val = X_val.reshape(len(X_val), reshape_form)

## ANN Model

In [5]:
visible = Input(shape=(13, ))
hidden1 = Dense(16, activation='relu')(visible)
hidden2 = Dense(8, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)
model = Model(inputs=visible, outputs=output)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 16)                224       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 369
Trainable params: 369
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

callback = EarlyStopping(monitor='accuracy', patience=10)

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=50,
                    verbose=2,
                    validation_data=(X_val, Y_val),
                    callbacks=[callback])

In [9]:
model.save('ANN_83_15.h5')

In [10]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Evaluation loss:', score[0])
print('Evaluation accuracy:', score[1])

Evaluation loss: 0.38373175263404846
Evaluation accuracy: 0.8315789699554443
